In [1]:
#!pip install selenium
#!pip install scrapy

In [2]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import random

In [3]:
driver = webdriver.Chrome(r'D:\UMD Master Year 2 College Work\Fall 2022-2023\INST737-IM01\chromedriver')
url = 'https://www.imdb.com/title/tt2527336/reviews?ref_=tt_ov_rt'
randomNumber = random.randint(0,4)
time.sleep(1)
driver.get(url)
time.sleep(1)
print(driver.title)
time.sleep(1)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.END)
time.sleep(randomNumber)
# body.send_keys(Keys.PAGE_DOWN)
# time.sleep(1)
# body.send_keys(Keys.PAGE_DOWN)
# time.sleep(1)
# body.send_keys(Keys.PAGE_DOWN)

Star Wars: Episode VIII - The Last Jedi (2017) - Star Wars: Episode VIII - The Last Jedi (2017) - User Reviews - IMDb


In [4]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [5]:
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
        body.send_keys(Keys.END)
        time.sleep(randomNumber)
    except:
        pass

100%|██████████| 283/283 [23:16<00:00,  4.93s/it]


In [6]:
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
first_review = reviews[0]
sel2 = Selector(text = first_review.get_attribute('innerHTML'))
rating = sel2.css('.rating-other-user-rating span::text').extract_first().strip()

In [7]:
review = sel2.css('.text.show-more__control::text').extract_first().strip()
review_date = sel2.css('.review-date::text').extract_first().strip()
author = sel2.css('.display-name-link a::text').extract_first().strip()
review_title = sel2.css('a.title::text').extract_first().strip()
review_url = sel2.css('a.title::attr(href)').extract_first().strip()
#helpfulness = sel2.css('.actions.text-muted::text').extract_first().strip()
print('nRating:',rating)
print('nreview_title:',review_title)
print('nAuthor:',author)
print('nreview_date:',review_date)
print('nreview:',review)
#print('nhelpfulness:',helpfulness)

nRating: 3
nreview_title: I'm going to keep this short (unlike the film).
nAuthor: BA_Harrison
nreview_date: 24 December 2017
nreview: After George Lucas's execrable prequels and the mediocre The Force Awakens, I've learnt to keep my expectations low for any new Star Wars films in order to avoid total disappointment. Having just seen The Last Jedi, it appears that I haven't been keeping them low enough.


In [8]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN    
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN    
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

100%|██████████| 7086/7086 [05:37<00:00, 20.99it/s]


In [9]:
review_df

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,24 December 2017,BA_Harrison,3,I'm going to keep this short (unlike the film...,After George Lucas's execrable prequels and th...,/review/rw5308106/?ref_=tt_urv
1,16 December 2017,view_and_review,3,Not this Disappointed Since Jar Jar Binks\n,I love the Star Wars franchise. I was so anxio...,/review/rw5308106/?ref_=tt_urv
2,21 January 2018,gogoschka-1,None,My Issues With The Storytelling in The Last J...,"I didn't hate TLJ, but even if I completely ig...",/review/rw5308106/?ref_=tt_urv
3,19 April 2021,mn2000,1,An embarrassment for the Star Wars fans\n,"When I watch this movie, it only makes me reme...",/review/rw5308106/?ref_=tt_urv
4,17 December 2017,marc5477,1,What the hell was this?\n,Wow this is getting hammered by reviews and no...,/review/rw5308106/?ref_=tt_urv
...,...,...,...,...,...,...
7081,15 January 2018,delmay,None,The music was incredible\n,I love how the main music theme is based upon ...,/review/rw5308106/?ref_=tt_urv
7082,17 January 2018,harrisonjwilson,6,Lazy!\n,Episode 8 was a very interesting movie. It was...,/review/rw5308106/?ref_=tt_urv
7083,17 January 2018,danielenglish,6,Bye Bye miss American Pie\n,"No spoilers, but If you haven't seen The Last ...",/review/rw5308106/?ref_=tt_urv
7084,7 June 2020,Luke_Gilbey,6,A Bad Good Movie\n,I'd describe The Last Jedi as a bad good movie...,/review/rw5308106/?ref_=tt_urv


In [10]:
#7090 reviews

In [13]:
#review_df.to_csv(r'D:\UMD Master Year 2 College Work\Fall 2022-2023\INST737-IM01\INST737_Star_Wars_IMDB_StarWars8.csv')